In [ ]:
#Train and save the model

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Initialize model, loss, and optimizer
model = LeNet5()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

#Save the trained model to extract weights later
torch.save(model.state_dict(), "lenet5_mnist.pth")
print("Model trained and saved as lenet5_mnist.pth")

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 14.6MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 477kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.41MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.35MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Loss: 0.2162
Epoch [2/5], Loss: 0.0332
Epoch [3/5], Loss: 0.0521
Epoch [4/5], Loss: 0.0086
Epoch [5/5], Loss: 0.0015
Model trained and saved as lenet5_mnist.pth


In [4]:
#Test the Model's performance against the Test Dataset

import torch
import torchvision
import torchvision.transforms as transforms

model = LeNet5()
model.load_state_dict(torch.load("lenet5_mnist.pth", map_location=torch.device('cpu')))
model.eval()

transform = transforms.Compose([transforms.ToTensor()])
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

def test_accuracy(model, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Consider the class with highest probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

test_accuracy(model, testloader)

<ipython-input-4-d0e8a4a29162>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lenet5_mnist.pth", map_location=torch.device('cpu')))


Test Accuracy: 98.47%


In [3]:
# Store the Model's weights from each layer in Binary/Text format
# These files will be used in our C program later when we are implementing the model using C and CUDA

import torch
import torch.nn as nn
import numpy as np

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)  # (6, 1, 5, 5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)  # (16, 6, 5, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # (120, 256)
        self.fc2 = nn.Linear(120, 84)  # (84, 120)
        self.fc3 = nn.Linear(84, 10)  # (10, 84)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet5()
model.load_state_dict(torch.load("lenet5_mnist.pth", map_location=torch.device('cpu')))
model.eval()

# Function to save weights to a text file
def save_weights_txt(model, filename="weights.txt"):
    with open(filename, "w") as f:
        for name, param in model.named_parameters():
            f.write(f"{name} {param.numel()} ")  # Store layer name and size
            np.savetxt(f, param.detach().numpy().flatten(), fmt="%.6f")  # Save weights

# Function to save weights in a binary file
def save_weights_bin(model, filename="weights.bin"):
    with open(filename, "wb") as f:
        for name, param in model.named_parameters():
            np.array([param.numel()], dtype=np.int32).tofile(f)  # Store size
            param.detach().numpy().flatten().astype(np.float32).tofile(f)  # Store weights

save_weights_txt(model, "weights.txt")
save_weights_bin(model, "weights.bin")

<ipython-input-3-b62b37b217d0>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lenet5_mnist.pth", map_location=torch.device('cpu')))
